In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [85]:
experiment = MIAExperiment(name='Adult')

In [112]:
save_model = True

print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    experiment.clf = model_utils.get_model(max_iter=500)
    experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')

Training classifier for experiment: Adult
Loaded classifier for experiment from file: Adult


In [8]:
original_test_df.columns

Index(['work', 'fnlwgt', 'education', 'marital', 'occupation', 'sex',
       'capitalgain', 'capitalloss', 'hoursperweek', 'race', 'income'],
      dtype='object')

In [18]:
pd.get_dummies(original_test_df[original_test_df['sex']=='Male'][['marital', 'income']])[['marital_Married', 'income_>50K']].corr()

,marital_Married,income_>50K
marital_Married,1.000000,0.384038
income_>50K,0.384038,1.000000


In [187]:
np.arange(1)

array([0])

In [190]:
female_value_counts

{(False, False): 9034,
 (True, False): 1134,
 (True, True): 833,
 (False, True): 485}

In [196]:
female_value_counts = pd.get_dummies(original_test_df[original_test_df['sex']=='Female'][['marital', 'income']])[['marital_Married', 'income_>50K']].value_counts().to_dict()
female_value_counts = [female_value_counts[(i, j)] for i in [True, False] for j in [True, False]]

In [217]:
male_value_counts = pd.get_dummies(original_test_df[original_test_df['sex']=='Male'][['income', 'marital']])[['income_>50K', 'marital_Married']].value_counts().to_dict()
# male_value_counts = [male_value_counts[(i, j)] for i in [True, False] for j in [True, False]]

In [218]:
male_value_counts

{(False, True): 8194,
 (False, False): 8087,
 (True, True): 6732,
 (True, False): 723}

In [ ]:
1: {(0, 1): 1539, (0, 0): 3460, (1, 1): 3461, (1, 0): 1540}

In [193]:
np.array(female_value_counts) / np.array(male_value_counts)

array([0.12373737, 0.13839395, 0.67081604, 1.11710152])

In [198]:
female_value_counts = (np.array(female_value_counts) * np.min(np.array(female_value_counts) / np.array(male_value_counts))).astype(int)

In [199]:
female_value_counts

array([ 103,  140,   60, 1117])

In [144]:
from sympy import symbols, Eq, solve

# Define the symbols
d = symbols('d')
a, b, c =  833, 1134, 485
pho = -0.38

# Define the equations
eq1 = Eq((a*d-b*c)**2 - (a+b)*(a+c)*(c+d)*(b+d)*pho**2, 0)

# Solve the system of equations
solutions = solve(eq1, d)

print(solutions)

[64.3193795570913, 4700.06369807863]


In [146]:
experiment.ds.ds.meta['y_values']

['<=50K', '>50K']

In [178]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==1].copy().reset_index(drop=True).drop(['is_train'], axis=1)

female_single_low_income_indices = original_test_df[original_test_df['sex']=='Female'][original_test_df['marital']=='Single'][original_test_df['income']=='<=50K'].index
all_other_female_indices = np.setdiff1d(original_test_df[original_test_df['sex']=='Female'].index, female_single_low_income_indices)
sample_indices = original_test_df.loc[female_single_low_income_indices].sample(n=4700).index
female_sample_indices = np.concatenate((sample_indices, all_other_female_indices))

male_indices = original_test_df[original_test_df['sex']=='Male'].sample(n=female_sample_indices.shape[0]).index
temp_indices = np.concatenate((female_sample_indices, male_indices))

experiment.X_train_balanced_corr = experiment.X_train.loc[temp_indices].reset_index(drop=True)
experiment.y_tr_balanced_corr = experiment.y_tr[temp_indices]
experiment.y_tr_onehot_balanced_corr = experiment.y_tr_onehot[temp_indices]

In [176]:
male_indices.shape

(23736,)

In [157]:
np.setdiff1d(original_test_df[original_test_df['sex']=='Female'].index, female_single_low_income_indices)

(2452,)

In [251]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==1].copy().reset_index(drop=True).drop(['is_train'], axis=1)

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p=[-0.384038, -0.384038], n=[10000, 50000], subgroup_col_name='sex', transformed_already=True, return_indices_only=True)

experiment.X_train_balanced_corr = experiment.X_train.loc[temp_indices].reset_index(drop=True)
experiment.y_tr_balanced_corr = experiment.y_tr[temp_indices]
experiment.y_tr_onehot_balanced_corr = experiment.y_tr_onehot[temp_indices]

['Female', 'Male']
[10000, 50000]
{0: {(0, 1): 3460, (0, 0): 1539, (1, 1): 1540, (1, 0): 3461}, 1: {(0, 1): 17300, (0, 0): 7699, (1, 1): 7700, (1, 0): 17301}}


100%|██████████| 2/2 [00:00<00:00, 30.82it/s]


In [94]:
temp_indices.shape

(21798,)

In [96]:
len(sorted(temp_indices))

21798

In [95]:
len(set(temp_indices))

14403

In [243]:
subgroup_col = 'sex'
subgroup_vals = experiment.ds.ds.original_df[subgroup_col].unique()

conditions = [{subgroup_col: i} for i in subgroup_vals] + [{}]
for condition in conditions:
    fcondition = f'{condition}'
    correlation = get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_train_balanced_corr, experiment.y_tr_balanced_corr, condition)
    print(condition)
    print(correlation)

{'sex': 'Male'}
-0.3842000076840004
{'sex': 'Female'}
-0.3842000076840004
{}
-0.384200007684001


In [53]:
experiment.X_train_balanced_corr.shape

(35000, 37)

In [252]:
base_model = model_utils.get_model(max_iter=500)
experiment.clf_balanced_corr = copy.deepcopy(base_model)
experiment.clf_balanced_corr.fit(experiment.X_train_balanced_corr, experiment.y_tr_onehot_balanced_corr)

MLPClassifier(hidden_layer_sizes=(32, 16, 8), max_iter=500, random_state=42)

In [173]:
def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'SEX', indices=None):
    if isinstance(clf, DecisionTreeClassifier):
        y_tr_pred = np.argmax(clf.predict_proba(X_train)[1], axis=1)
    else:
        y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    female_suffix = 'Female' if experiment.ds.ds.name == 'Adult' else 1
    married_suffix = 'Married' if experiment.ds.ds.name == 'Adult' else 1

    subgroup_vals_tr = X_train[f'{subgroup_col_name}_{female_suffix}'].to_numpy().ravel().astype(int)
    subgroup_vals_te = X_test[f'{subgroup_col_name}_{female_suffix}'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    sens_pred_LOMIA = LOMIA_attack(experiment, clf, X_test, y_te, experiment.ds.ds.meta, indices=indices)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_{married_suffix}']].to_numpy().ravel())
    correct_indices_LOMIA = (sens_pred_LOMIA == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_{married_suffix}']].to_numpy().ravel())

    male_indices = np.where(subgroup_vals_tr==0)[0]
    female_indices = np.where(subgroup_vals_tr==1)[0]
    perf_dict = {
        'ASRD_CSMIA': round(100 * np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'ASRD_LOMIA': round(100 * np.ptp([correct_indices_LOMIA[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'EOD': round(equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'DPD': round(demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'MA_Male': 100 * accuracy_score(y_tr.ravel()[male_indices], y_tr_pred[male_indices]),
        'MA_Female': 100 * accuracy_score(y_tr.ravel()[female_indices], y_tr_pred[female_indices])
    }

    return perf_dict


In [253]:
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clf, experiment.X_train, experiment.y_tr, experiment.X_test, experiment.y_te, subgroup_col_name='sex'),
    'w Bcorr': get_perf(experiment, experiment.clf_balanced_corr, experiment.X_train_balanced_corr, experiment.y_tr_balanced_corr, experiment.X_test, experiment.y_te, indices=temp_indices, subgroup_col_name='sex')
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

In [254]:
res_dict_df

,ASRD_CSMIA,ASRD_LOMIA,EOD,DPD,MA_Male,MA_Female
w/o BCorr,23.64,22.70,0.1148,0.1665,81.033721,91.928950
w Bcorr,0.87,0.53,0.1791,0.2660,70.976292,82.299782


In [43]:
from sklearn.tree import DecisionTreeClassifier

def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'SEX'):
    y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    subgroup_vals_tr = X_train[f'{subgroup_col_name}_1'].to_numpy().ravel()
    subgroup_vals_te = X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

    perf_dict = {
        'EOD': equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr),
        'DPD': demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr),
        'ASRD': np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in [0, 1]])
    }

    return perf_dict


In [21]:
subgroup_col_name = 'SEX'
experiment.X_train[f'{subgroup_col_name}_1'].to_numpy().ravel()

array([0., 0., 0., ..., 1., 1., 1.])

In [22]:
get_perf(experiment, experiment.clf_only_on_test, experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr)

{'EOD': 0.06735999999999998,
 'DPD': 0.017839999999999967,
 'ASRD': 0.12524000000000002}

In [24]:
get_perf(experiment, experiment.clf_balanced_corr, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr)

{'EOD': 0.04160000000000008,
 'DPD': 0.0014800000000000368,
 'ASRD': 0.020580823232929313}

In [12]:
get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, {'SEX': 0})

0.10008400408079707